# 1. All Imports
![image.png](attachment:image.png)

In [1]:
# Import torch and nn
import torch
from torch import nn
from torch.nn import functional as F

# 2. Assign Hyper Parameters

In [2]:
# hyperparameters
batch_size = 32 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?
max_iters = 3000
eval_interval = 300
learning_rate = 1e-2
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200

# 3. Upload wizard_of_oz.txt from the repo, you are being shared

In [3]:
from google.colab import files
uploaded = files.upload()

with open('./wizard_of_oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()

Saving wizard_of_oz.txt to wizard_of_oz (1).txt


## 4. Words to integer Conversion
![image.png](attachment:image.png)

In [4]:
# Create a encode and decode function
chars = sorted(list(set(text)))
vocab_size = len(chars)

stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string


## 5. Split Train and Test Data

![image.png](attachment:image.png)

In [5]:
# 90% training and 10% validation split

data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]


## 6. Batch data and stack them for prediction
![image.png](attachment:image.png)

In [6]:
# Split data as input and target z, z+1 -> x, y respectively

def get_batch(split):
  data = train_data if split == 'train' else val_data
  ix = torch.randint(len(data) - block_size, (batch_size,))
  x = torch.stack([data[i:i+block_size] for i in ix])
  y = torch.stack([data[i+1:i+block_size+1] for i in ix])
  x, y = x.to(device), y.to(device)
  return x, y


## 7. Observe accuracy so we can measure on every few steps

In [7]:
# Function to estimate loss and run on every learning iteration
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

## 8. Complete BigramLM

![image.png](attachment:image.png)

In [8]:
# Create BigramLanguage Model class
class BigramLanguageModel(nn.Module):

  def __init__(self, vocab_size) -> None:
      super().__init__()
      self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

  def forward(self, idx, targets=None):
    logits = self.token_embedding_table(idx)

    if targets is None:
      loss = None
    else:
      B, T, C = logits.shape
      logits = logits.view(B*T, C)
      targets = targets.view(B*T)
      loss = F.cross_entropy(logits, targets)

    return logits, loss

  def generate(self, idx, max_new_tokens):
    for _ in range(max_new_tokens):
      logits, loss = self(idx)
      logits = logits[:, -1, :]
      probs = F.softmax(logits, dim=-1)
      idx_next = torch.multinomial(probs, num_samples=1)
      idx = torch.cat((idx, idx_next), dim=1)
    return idx



## 9. Run the model and assign an optimizer

In [9]:
# Run the model to get the output
model = BigramLanguageModel(vocab_size)
m = model.to(device)

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

## 10. Run deep learning training to learn based max_iters

In [10]:
# Training loop
for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

step 0: train loss 4.7908, val loss 4.8106
step 300: train loss 2.8018, val loss 2.8437
step 600: train loss 2.5288, val loss 2.5735
step 900: train loss 2.4703, val loss 2.5239
step 1200: train loss 2.4642, val loss 2.4929
step 1500: train loss 2.4541, val loss 2.4947
step 1800: train loss 2.4504, val loss 2.5001
step 2100: train loss 2.4448, val loss 2.4829
step 2400: train loss 2.4417, val loss 2.4805
step 2700: train loss 2.4358, val loss 2.4819


## 11. Final step to get output from trained model

In [11]:
# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=500)[0].tolist()))



su r a cl&en's rn gio beanoted he d om nors llkitcokiman henerd ge﻿13?"tier he-HAnthe ourd de
P-y
aind heys inove iNowlof jucaimasharered thewhetout?" t, any there henss t o he tre roofouit Weas ware p icoremid apaconsunde memotouresiversco tt e, aro awacanesharobrin; t. tt idy.
"I  se kebe Linee ve t ck e gryt skaldlfow l amo lavestot  suthe l HE4  Jisk, waso s  w aies. thegsthe eatat't otlK Gldim, an gharcesind Zedie the ganthofrd e h, Jilovere D oned we m, plar



" wlime he we
sthe using do
